# Leaflet cluster map of talk locations

The following assumes you are working with a clean installation of Linux or Windows Subsystem for Linux environment:

```bash
# Install dependencies
sudo apt install jupyter python3-pip
pip install python-frontmatter getorg --upgrade
```
Navigate to the `_talks/` directory and run the following command to generate the `talkmap_out.ipynb` notebook:

```bash
# Generate the talkmap
jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```

This notebook assumes that your presentations in the `_talks/` directory are stored as markdown files. This code scrapes the location YAML field from each file with the `.md` extension, uses `geophy/Nominatim` for geolocation and the `getorg` library to ouput data, HTML, and JavaScript for a standalone cluster map.

In [ ]:
# Start by installing the dependencies
%pip install python-frontmatter getorg --upgrade
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

In [ ]:
# Collect the Markdown files
g = glob.glob("*.md")

In [ ]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="academicpages.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

If timeout error occurs, verify the location can be properly geolocated.

In [ ]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    if 'location' not in data:
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as error:
        print(f"Error: geocode failed on input {location} with messaged {error}")
    except GeocoderTimedOut as error:
        print(f"Error: geocode timed out on input {location} with messaged {error}")
    except Exception as Error:
        print(f"An unhandled exception occured while processing input {location} with message {error}")

In [ ]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)